In [8]:
import pandas as pd
import os
import json
from datetime import date
import datetime
now =  datetime.datetime.now()

In [1]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import secret

api_key = secret.api_key
api_secret = secret.api_secret

kite = KiteConnect(api_key=api_key)

In [2]:
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=x5ugrx91yexerba1&v=3


In [3]:
data = kite.generate_session("NuISoeflbRJshBeEPRh6U39HhogdyNDo", api_secret=api_secret)
kite.set_access_token(data["access_token"])
access_token = data["access_token"]
kws = KiteTicker(api_key, access_token)
print(data["access_token"])

RuQ5t5goLFnT6zy9SUNiLOO9GEjVLJIK


In [4]:
def find_neighbour(sorted_data,compare_num):
    lower_neighbors = []
    higher_neighbors = []
    
    for num in sorted_data:
        if num < compare_num:
            lower_neighbors.append(num)
        elif num > compare_num:
            higher_neighbors.append(num)

    # Get first, second, and third lower neighbors (if available)
    first_lower = lower_neighbors[-1] if len(lower_neighbors) > 0 else None
    second_lower = lower_neighbors[-2] if len(lower_neighbors) > 1 else None
    third_lower = lower_neighbors[-3] if len(lower_neighbors) > 2 else None

    # Get first, second, and third higher neighbors (if available)
    first_higher = higher_neighbors[0] if len(higher_neighbors) > 0 else None
    second_higher = higher_neighbors[1] if len(higher_neighbors) > 1 else None
    third_higher = higher_neighbors[2] if len(higher_neighbors) > 2 else None

    return first_lower,second_lower,third_lower,first_higher,second_higher,third_higher


In [5]:
def percentage(close_price,percent_value):
    percnt = (close_price/100)*percent_value
    addprcntge = close_price+percnt
    subprcntge = close_price-percnt
    return addprcntge,subprcntge

In [6]:
def comparingLtp(stockLTP,neighbourList):
    stockList = []
    
    addpercentage,subpercentage = percentage(stockLTP,2)
    
    for num in neighbourList:
        try:
            if subpercentage <= num <= addpercentage:
                stockList.append(num)
        except: continue
    if stockList: 

        return stockList

In [ ]:
watchlist = "Nifty200.csv"
timeframe = "M"
stock_dict = dict()
stock =  pd.read_csv(f"watchList\\{watchlist}")
leveltap = pd.read_csv(f"otherImp\\{timeframe}\\{watchlist}")
instruments = kite.instruments("NSE")
symbols = stock['Symbol']
count = 0 
for symbol in symbols:
    
    to_date = date.today()
    from_date = "2019-10-01"
    try:
        instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbol)
        data = kite.historical_data(instrument_token,from_date,to_date,"day")
        df = pd.DataFrame(data)

        df['date'] = pd.to_datetime(df['date']).dt.to_period('M') # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
        monthly_data = df.groupby('date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).reset_index()

        last_month = len(monthly_data)-2
        close_data = monthly_data['close'][last_month]
        level = leveltap['levels'][count]
        count+=1
        data = json.loads(level)
        sorted_data =  sorted(map(float,data.keys()))
       
        first_lower,second_lower,third_lower,first_higher,second_higher,third_higher = find_neighbour(sorted_data,close_data)  
        stock_dict[symbol] =  [first_lower,second_lower,third_lower,first_higher,second_higher,third_higher,close_data]
    except:continue

if stock_dict:
    df = pd.DataFrame.from_dict(stock_dict,orient='index').reset_index()
    df.columns = ['stocks','first_lower','second_lower','third_lower','first_higher','second_higher','third_higher','close_data']
    df.to_csv(f"MonthlyCloseLevels\\{timeframe}\\{watchlist}",index=False)
    

In [22]:
def monthlylevels(timeframe,watchlist):
    stockDict = dict()
    watchlist = f"MonthlyCloseLevels\\{timeframe}\\{watchlist}"
    stockname =  pd.read_csv(watchlist)
    count= 0
    for index,symbol in stockname.iterrows():
        ltp = kite.ltp('NSE:'+symbol['stocks'])
        stockLTP = ltp['NSE:'+symbol['stocks']]['last_price']
   
        stockList = comparingLtp(stockLTP,[symbol['second_lower'],symbol['third_lower'],symbol['second_higher'],symbol['third_higher']])
    
        if stockList:
            stockDict[symbol['stocks']] = [stockLTP, stockList, float(symbol['close_data']),[symbol['first_lower'],symbol['second_lower'],symbol['third_lower'],symbol['first_higher'],symbol['second_higher'],symbol['third_higher']]] 
    if stockDict:
        df = pd.DataFrame.from_dict(stockDict,orient='index').reset_index()
        
        df.columns = ['stocks','ltp','taps','closedata','levels']
        df['date'] = date.today()
        df['time'] =  now.strftime("%Y%m%d_%H%M%S")
        try:
            df.to_excel(f"monthlyRev\\monthlyRev_{now.strftime('%Y%m%d_%H%M%S')}.xlsx",index=False)
            print('Done')
        except:
            print('Error')
        


In [23]:
def submit_form():
    watchlist = watchlist_var.get()
    timeframe = timeframe_var.get()
    monthlylevels(timeframe,watchlist)
    
import tkinter as tk
from tkinter import ttk
import os

watchListDirec = "watchList"
li=[]

for name in os.listdir(watchListDirec):
    li.append(name)

# Create the main window
root = tk.Tk()
root.title("Trading Watchlist Form")

# Watchlist Label and Dropdown
watchlist_label = ttk.Label(root, text="Select Watchlist:")
watchlist_label.pack(pady=5)
watchlist_var = tk.StringVar()
watchlist_menu = ttk.Combobox(root, textvariable=watchlist_var, values=li)
watchlist_menu.pack(pady=5)

# Timeframe Label and Dropdown
timeframe_label = ttk.Label(root, text="Select Timeframe:")
timeframe_label.pack(pady=5)
timeframe_options = ["M","W"]
timeframe_var = tk.StringVar(value=timeframe_options[0])
timeframe_menu = ttk.Combobox(root, textvariable=timeframe_var, values=timeframe_options)
timeframe_menu.pack(pady=5)

# Submit Button
submit_button = ttk.Button(root, text="Submit", command=submit_form)
submit_button.pack(pady=20)

# Run the main loop
root.mainloop()

Done
